<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Initialization" data-toc-modified-id="Initialization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialization</a></span></li><li><span><a href="#Some-helper-functions" data-toc-modified-id="Some-helper-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Some helper functions</a></span></li><li><span><a href="#Build-a-model-with-a-variable-number-of-layers" data-toc-modified-id="Build-a-model-with-a-variable-number-of-layers-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Build a model with a variable number of layers</a></span></li><li><span><a href="#Data-Input" data-toc-modified-id="Data-Input-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Input</a></span></li><li><span><a href="#Preprocess-Data" data-toc-modified-id="Preprocess-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Preprocess Data</a></span></li><li><span><a href="#Logistic-Regression-as-a-Keras-Model" data-toc-modified-id="Logistic-Regression-as-a-Keras-Model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Logistic Regression as a Keras Model</a></span></li><li><span><a href="#Keras-Model" data-toc-modified-id="Keras-Model-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Keras Model</a></span></li><li><span><a href="#Training-the-model" data-toc-modified-id="Training-the-model-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Training the model</a></span></li><li><span><a href="#Evaluating-the-model" data-toc-modified-id="Evaluating-the-model-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Evaluating the model</a></span><ul class="toc-item"><li><span><a href="#Training-Data" data-toc-modified-id="Training-Data-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Training Data</a></span></li><li><span><a href="#Test-Data" data-toc-modified-id="Test-Data-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Test Data</a></span></li></ul></li><li><span><a href="#Classification-quality-for-different-network-architectures" data-toc-modified-id="Classification-quality-for-different-network-architectures-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Classification quality for different network architectures</a></span><ul class="toc-item"><li><span><a href="#One-hidden-layer" data-toc-modified-id="One-hidden-layer-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>One hidden layer</a></span></li><li><span><a href="#Two-hidden-layers" data-toc-modified-id="Two-hidden-layers-10.2"><span class="toc-item-num">10.2&nbsp;&nbsp;</span>Two hidden layers</a></span></li><li><span><a href="#Multiple-hidden-layers" data-toc-modified-id="Multiple-hidden-layers-10.3"><span class="toc-item-num">10.3&nbsp;&nbsp;</span>Multiple hidden layers</a></span></li></ul></li></ul></div>

# Classification with Neural Networks implemented with Keras
This notebook illustrates how to implement a neural network classifier with TensorFlow.  
We're using the dataset from the
[Bank Marketing Dataset from the UCI Data Repository](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing)

## Initialization

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# turn off tensorflow deprecation warnings
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from IPython.display import SVG
from keras.utils import model_to_dot

## Some helper functions

This function displays a few quality measure for the classifier
* confusion matrix
* classification accuracy
* area under curve (AUC)

In [ ]:
def quality(model, features, labels):
    predictions = model.predict_classes(features)
    conf_matrix = confusion_matrix(labels, predictions)   
    _, accuracy = model.evaluate(features, labels, verbose=0)  
    probabilities = model.predict(features)
    auc = roc_auc_score(labels, probabilities)
    return conf_matrix, accuracy, auc

Plot the history of the model fit. The Keras `Model.fit` function returns a history object which  is a record of training loss values and metrics values at successive epochs, as well as validation loss values and validation metrics values. Plotting these gives a good idea about possible overfitting.

In [ ]:
def plot_history(history):
    # to handle metrics keys changes in Keras 2.3
    # see https://github.com/keras-team/keras/releases/tag/2.3.0
    pre_23 = "acc" in history.history.keys()
    acc_key     = 'acc'     if pre_23 else 'accuracy'
    val_acc_key = 'val_acc' if pre_23 else 'val_accuracy'       
    
    acc = history.history[acc_key]
    val_acc = history.history[val_acc_key]
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    # "bo" is for "blue dot"
    plt.plot(epochs, loss, 'bo', label='Training loss')
    # b is for "solid blue line"
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

## Build a model with a variable number of layers
We're using a simple Keras `Sequential` model which is a linear stack of network layers. For the input and intermediate layers we're using the `ReLU` and for the final layer the `sigmoid` activation function.

Obviously, there are lots of hyperparameters you can play with. This little helper just makes it easy to define models with different depths. The `layers` parameter is an array of integers each of which defines the number of nodes for a particular layer.

In [ ]:
def make_model(num_inputs, layers):
    model = Sequential()

    # first layer
    model.add(Dense(layers[0], input_dim=num_inputs, activation='relu'))

    #  intermediate layers
    for i in range(1, len(layers)):
        model.add(Dense(layers[i], activation='relu'))
    
    # final layer with a single node
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    
    return model

## Data Input
Read the bank data set and split into a features and a label subset

In [ ]:
data_sets = ('bank-10percent', 'bank-full', 'bank-balanced')

bank = pd.read_csv('../data/' + data_sets[2] + '.csv')

label_col = 'y'
label = bank[label_col]
features = bank.drop(columns=['y'])

label_encoded = pd.get_dummies(label, drop_first = True)
features_encoded = pd.get_dummies(features, drop_first = True)
feature_count=features_encoded.shape[1]

## Preprocess Data
It's recommended to normalize the data

In [ ]:
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features_encoded)

Split into training and test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_normalized, label_encoded, test_size = 0.2, random_state = 167)

## Logistic Regression as a Keras Model
As a benchmark create and evaluate a network without any hidden layer and a `sigmoid` activation function. This corresponds to Logistic Regression classifier.

In [ ]:
logistic_model = Sequential()
logistic_model.add(Dense(1, input_dim=feature_count, activation='sigmoid')) 
logistic_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
logistic_model.fit(X_train, y_train, epochs=20)
conf_matrix, accuracy, auc = quality(logistic_model, X_test, y_test)
print("\nConfusion Matrix:\n{0}".format(conf_matrix))
print("\nAccuracy: {0:.2f} %".format(accuracy*100), "AUC: {0:.3f} %".format(auc))

## Keras Model
This is our model : it has two hidden layers with 16 and 8 nodes, respectively.

In [ ]:
model = make_model(feature_count, [16, 8])

Let's output a bit more information about the model. Note that large number of parameters to be trained.

In [ ]:
print("Number of input features : {0}".format(feature_count))
print(model.summary())

We can also draw a picture showing the network topology.

In [ ]:
SVG(model_to_dot(model, show_shapes=True, show_layer_names=False).create(prog='dot', format='svg'))

## Training the model
When training the model we have to specify the number of epochs. An `epoch` is an iteration over the entire training set. However, gradient updates are not done using the entire set but batches of training data instead. The default batch size is 32, i.e. an iteration involves (training set size)/32 gradient updates. 

In [ ]:
num_epochs=50
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=num_epochs, 
                    verbose=1)

Now let's plot the loss function during optimization.

In [ ]:
plot_history(history)

Notice that the training loss keeps decreasing but the validation error doesn't decrease any more after about 10 epochs. In fact, it even increases afterwards. This indicates that the models begins to overfit.

## Evaluating the model
### Training Data

In [ ]:
conf_matrix, accuracy, auc = quality(model, X_train, y_train)
print("\nConfusion Matrix:\n{0}".format(conf_matrix))
print("\nAccuracy: {0:.2f} %".format(accuracy*100), "AUC: {0:.3f} %".format(auc))

### Test Data

In [ ]:
conf_matrix, accuracy, auc = quality(model, X_test, y_test)
print("\nConfusion Matrix:\n{0}".format(conf_matrix))
print("\nAccuracy: {0:.2f} %".format(accuracy*100), "AUC: {0:.3f} %".format(auc))

## Classification quality for different network architectures
We vary the number of nodes in the hidden layers. Note that the classification quality doesn't really change. This is not particularly surprising, since the logistic regression (which is equivalent to an NN without any hidden layer) already achieves a very good result.

In [ ]:
def train_and_evaluate(num_inputs, layers, X_train, y_train, X_test,y_test):
    model = make_model(num_inputs, layers)
    print(model.summary())
    model.fit(X_train, y_train,
              epochs=10, 
              verbose=0)
    _, accuracy, auc = quality(model, X_test,y_test)
    print("\nAccuracy: {0:.2f} %".format(accuracy*100), "AUC: {0:.3f} %".format(auc))

### One hidden layer

In [ ]:
train_and_evaluate(feature_count, [16], X_train, y_train, X_test,y_test)

### Two hidden layers

In [ ]:
train_and_evaluate(feature_count, [16, 8], X_train, y_train, X_test,y_test)

### Multiple hidden layers
Finally check whether adding multiple layers makes any difference

In [ ]:
train_and_evaluate(feature_count, [32, 16, 8, 4], X_train, y_train, X_test,y_test)